In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from clawpack import riemann
import numpy as np

# Riemann solver animation
The next cell sets up a function that takes a Riemann problem (left state, right state, and approximate solver) and animates the approximate solution.

In [ ]:
from matplotlib import animation
from clawpack.visclaw.JSAnimation import IPython_display

def riemann_solution(num_eqn,solver,q_l,q_r,aux_l=None,aux_r=None,t=0.2,problem_data=None):
    if aux_l is None:
        aux_l = np.zeros(num_eqn)
    if aux_r is None:
        aux_r = np.zeros(num_eqn)
    
    wave, s, amdq, apdq = solver(q_l.reshape((num_eqn,1)),q_r.reshape((num_eqn,1)),
                                 aux_l.reshape((num_eqn,1)),aux_r.reshape((num_eqn,1)),problem_data)
    
    x = np.linspace(-1.,1.,1000)
    
    states = [q_l]
    for i in range(len(s)):
        states.append(states[-1] + np.squeeze(wave[:,i,0]))
    
    fig = plt.figure(figsize=(18,6))
    axes = [0]*num_eqn
    lines = [0]*num_eqn
    for i in range(num_eqn):
        axes[i] = fig.add_subplot(1,num_eqn,i+1)
        lines[i], = plt.plot(x,0*x,lw=2)
        axes[i].set_xlim((-1,1))
        qmax = max([state[i] for state in states])
        qmin = min([state[i] for state in states])
        qdiff = qmax - qmin
        axes[i].set_ylim((qmin-0.1*qdiff,qmax+0.1*qdiff))
        plt.title('q['+str(i)+']')
    
    if num_eqn == 2:
        q0 = np.array([state[0] for state in states])
        q1 = np.array([state[1] for state in states])
        fig2 = plt.figure()
        plt.plot(q0,q1,'o-k')
        plt.title('phase plane')
        plt.axis('equal')
 
    def fplot(t):
        q = np.outer(states[0],(x<t*s[0]))
    
        for i in range(len(s)-1):
            q = q + np.outer(states[i+1],(x>=t*s[i])*(x<t*s[i+1]))
        q = q + np.outer(states[-1],(x>s[-1]*t))
    
        for i in range(num_eqn):
            lines[i].set_data(x,q[i,:])
        
        return lines,
    
    return animation.FuncAnimation(fig, fplot, frames=np.linspace(0.,1.,11), interval=300)

# Acoustics: exact solution
We can use this to examine the exact solution of an acoustics Riemann problem:

In [ ]:
solver = riemann.acoustics_1D_py.acoustics_1D
num_eqn = riemann.acoustics_1D_py.num_eqn

q_l = np.array((1,4))
q_r = np.array((3,7))

problem_data = {}
problem_data['zz'] = 1.0
problem_data['cc'] = 1.0

riemann_solution(num_eqn,solver,q_l,q_r,problem_data=problem_data)

# Euler: approximate solutions
We can also easily compare the approximate solutions given by, say, a Roe solver and an HLL solver for the Euler equations.

In [ ]:
solver = riemann.euler_1D_py.euler_roe_1D
num_eqn = riemann.euler_1D_py.num_eqn

q_l = np.array((3.,0.,3.))
q_r = np.array((1.,0.,1.5))

problem_data = {}
problem_data['gamma'] = 1.4
problem_data['gamma1'] = 1.4-1.0
problem_data['efix'] = False

riemann_solution(num_eqn,solver,q_l,q_r,problem_data=problem_data)

In [ ]:
solver = riemann.euler_1D_py.euler_hll_1D
riemann_solution(num_eqn,solver,q_l,q_r,problem_data=problem_data)